In [2]:
import gurobipy as gp
import pandas as pd
from code_map import final_markets, new_meters, utils, data_handling
import numpy as np
from datetime import datetime, timedelta



NameError: name 'Utils' is not defined

In [ ]:
timeframe = utils.half_month
L, M, F, H, freq_data, power_meter_dict, consumption_data = utils.get_all_sets(timeframe)


In [ ]:
L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m = utils.get_parameters(L,M,H)

In [ ]:
markets_name_dict = {market.name: market for market in M}
market_names = list(markets_name_dict.keys())

In [ ]:
dominant_directions = [utils.get_dominant_direction(freq_data, hour) for hour in H]
Ir_hlm, Ia_hlm, Va_hm = utils.get_income_dictionaries(H, M, L, dominant_directions, Fu_h_l, Fd_h_l, P_h_m, Vp_h_m, F, markets_name_dict, timeframe=timeframe)


In [13]:
compatible_list = utils.get_compatibility_list(H=H, L=L, M=M)


In [ ]:

def find_previous_weekday(d, weekday):
    days_behind = d.weekday() - weekday
    if days_behind < 0: # Target day is in the previous week
        days_behind += 7
    return d - timedelta(days=days_behind)

# Function to calculate the deadlines for a given input timestamp
def calculate_deadlines_for_input(input_timestamp):
    deadlines = {}

    # RKOM uke: Friday the week before if on a weekday, otherwise Thursday the same week
    if input_timestamp.weekday() < 5:  # Weekday
        deadlines['RKOM uke'] = find_previous_weekday(input_timestamp - timedelta(days=7), 4)  # Previous Friday
    else:  # Weekend
        deadlines['RKOM uke'] = find_previous_weekday(input_timestamp, 3)  # Current Thursday

    # FCR D-2: Two days before the input time at 17:30
    deadlines['FCR D-2'] = datetime.combine(input_timestamp.date() - timedelta(days=2), datetime.strptime('17:30', '%H:%M').time())

    # aFRR CM: One day before the input time at 07:30
    deadlines['aFRR CM'] = datetime.combine(input_timestamp.date() - timedelta(days=1), datetime.strptime('07:30', '%H:%M').time())

    # FCR D-1: One day before the input time at 18:00
    deadlines['FCR D-1'] = datetime.combine(input_timestamp.date() - timedelta(days=1), datetime.strptime('18:00', '%H:%M').time())

    # RK: 45 minutes before the input time
    deadlines['RK'] = input_timestamp - timedelta(minutes=45)

    return deadlines

# Example usage of the function
example_input_timestamp = datetime(2023, 12, 13, 15, 0)  # Example: 13th December 2023, 15:00
deadlines = calculate_deadlines_for_input(example_input_timestamp)
for market, deadline in deadlines.items():
    print(f"{market}: {deadline.strftime('%Y-%m-%d %H:%M')}")
